In [1]:
import kaggle

In [2]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
new_file= zipfile.ZipFile('orders.csv.zip')
new_file.extractall()
new_file.close()

In [4]:
import pandas as pd
df=pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
df.head(5)
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [5]:
df.columns

Index(['Order Id', 'Order Date', 'Ship Mode', 'Segment', 'Country', 'City',
       'State', 'Postal Code', 'Region', 'Category', 'Sub Category',
       'Product Id', 'cost price', 'List Price', 'Quantity',
       'Discount Percent'],
      dtype='object')

##Cleaning the dataset

In [7]:
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')

In [8]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [9]:
df['discount']=(df['list_price']*df['discount_percent'])/100
df['sell_price']=df['list_price']-df['discount']
df['profit']= df['sell_price']-df['cost_price']

drop unnecessary column adn convert date format`

In [11]:
df.dtypes
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [12]:
df.drop(columns=['cost_price','list_price','discount_percent'], inplace=True)

In [13]:
df


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sell_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,2,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,5,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,2,1.0,19.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,2023-02-18,Second Class,Consumer,United States,Miami,Florida,33180,South,Furniture,Furnishings,FUR-FU-10001889,3,1.2,28.8,-1.2
9990,9991,2023-03-17,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Furniture,Furnishings,FUR-FU-10000747,2,3.6,86.4,16.4
9991,9992,2022-08-07,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Technology,Phones,TEC-PH-10003645,2,5.2,254.8,34.8
9992,9993,2022-11-19,Standard Class,Consumer,United States,Costa Mesa,California,92627,West,Office Supplies,Paper,OFF-PA-10004041,4,0.9,29.1,-0.9


In [14]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="prakash"
uname="root"
pwd="MyNewPass"
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

# df.to_sql(name='df_orders', con=engine, index=False,if_exists='replace')

df.to_sql('df_orders', con=engine, index=False,if_exists='append')

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'df_orders.PRIMARY'")
[SQL: INSERT INTO df_orders (order_id, order_date, ship_mode, segment, country, city, state, postal_code, region, category, sub_category, product_id, quantity, discount, sell_price, profit) VALUES (%(order_id)s, %(order_date)s, %(ship_mode)s, %(segment)s, %(country)s, %(city)s, %(state)s, %(postal_code)s, %(region)s, %(category)s, %(sub_category)s, %(product_id)s, %(quantity)s, %(discount)s, %(sell_price)s, %(profit)s)]
[parameters: [{'order_id': 1, 'order_date': datetime.datetime(2023, 3, 1, 0, 0), 'ship_mode': 'Second Class', 'segment': 'Consumer', 'country': 'United States', 'city': 'Henderson', 'state': 'Kentucky', 'postal_code': 42420, 'region': 'South', 'category': 'Furniture', 'sub_category': 'Bookcases', 'product_id': 'FUR-BO-10001798', 'quantity': 2, 'discount': 5.2, 'sell_price': 254.8, 'profit': 14.800000000000011}, {'order_id': 2, 'order_date': datetime.datetime(2023, 8, 1